# Розвиток громадських організацій через призму активності населення

---

## Підготовка пакетів

In [90]:
#install.packages("NbClust")

In [2]:
library(tidyverse)

In [15]:
Sys.setlocale("LC_CTYPE", "ukrainian") 
options(warn = -1)

[1] "Ukrainian_Ukraine.1251"

In [16]:
getwd()

[1] "e:/Repos/Season_2022/paper-2022-09-survey"

## Завантаження даних

In [6]:
survey_data <- read.csv("survey_data.csv", encoding = "UTF-8") |> dplyr::select(-1)

In [7]:
head(survey_data)

,X1..Ваш.вік.,X2..Ваша.стать.,X3..Сфера.вашої.діяльності.,X4..Чи.цікавились.Ви.діяльністю.громадських.організацій.,X5..На.скільки.добре.Ви.обізнані.в.діяльності.громадських.організацій.,X6..Чи.приймали.Ви.участь.в.громадській.організації.,X7..Якщо.на.питання.6.Ви.відповіли.ТАК.тоді..якого.типу.це.була.організація.,X8..Якщо.на.питання.6.Ви.відповіли.НІ.тоді.вкажіть.причини.,X9..На.даний.момент..чи.є.Ви.членом.громадської.організації.,X10..Чи.приймали.Ви.участь.в.одноразовій.акції.,X11..Чи.є.хтось.з.Ваших.знайомих.членом.громадської.організацій.,X12..В.організації.якого.типу.Ви.б.захотіли.прийняти.участь...якщо.б.у.Вас.був.час.і.можливість.,X13..На.Вашу.думку..чи.є.в.Україні.потреба.в.створенні.значної.кількості.громадських.організацій.,X14..На.вашу.думку..чи.варто.проводити.більш.маштабні.акції.агітації..для.залучення.більшої.кількості.людей.
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,16-23 років,Жіноча,Працюючий,"Так, інколи","Орієнтуюсь, приймав/ла участь в одній організації",Так,Молодіжна організація,,Ні,Так,"Так, декілька",Творчі об'єднання,"Так, оскільки за допомогою них можна реалізувати себе","Так, за умови відсутності пропаганди діяльності цієї організації"
2,16-23 років,Жіноча,Студент,"Так, інколи","Не сильно, щось чув/ла, але не цікавився/лася.",Ні,,,Ні,Ні,"Ні, жодного",Молодіжна організація,"Так, оскільки вони покликані допомагати",Якщо це законно і відстоює благодійні справи
3,16-23 років,Жіноча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Так,Молодіжна організація,,Ні,Так,"Ні, жодного",Молодіжна організація,"Так, оскільки за допомогою них можна реалізувати себе","Так, за умови відсутності пропаганди діяльності цієї організації"
4,16-23 років,Жіноча,Студент,"Так, часто","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Ні,,,Ні,Ні,"Так, декілька",Молодіжна організація,"Так, оскільки вони покликані допомагати",Так
5,16-23 років,Жіноча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Ні,,Не мав/ла часу,Ні,Ні,"Ні, жодного",Творчі об'єднання,"Так, оскільки за допомогою них можна реалізувати себе",Можливо
6,16-23 років,Жіноча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Ні,,Не мав/ла змоги,Ні,Так,"Так, декілька",Молодіжна організація,"Так, оскільки вони покликані допомагати",Так


## Очистка даних

Відокремимо запитання у окрему таблицю

In [8]:
survey_headers <- data.frame(Variables = paste("X", 1:14, sep=""), Questions = colnames(survey_data))
survey_headers$Questions <- gsub("[.]", " ", survey_headers$Questions)
survey_headers$Questions <- substring(survey_headers$Questions, 4, last = 1000000L)
survey_headers

Variables,Questions
<chr>,<chr>
X1,Ваш вік
X2,Ваша стать
X3,Сфера вашої діяльності
X4,Чи цікавились Ви діяльністю громадських організацій
X5,На скільки добре Ви обізнані в діяльності громадських організацій
X6,Чи приймали Ви участь в громадській організації
X7,Якщо на питання 6 Ви відповіли ТАК тоді якого типу це була організація
X8,Якщо на питання 6 Ви відповіли НІ тоді вкажіть причини
X9,На даний момент чи є Ви членом громадської організації


Переіменуємо стовпці для зручнішого використання у коді даних

In [9]:
colnames(survey_data) <- survey_headers$Variables
head(survey_data[1:3,1:5])

,X1,X2,X3,X4,X5
,<chr>,<chr>,<chr>,<chr>,<chr>
1,16-23 років,Жіноча,Працюючий,"Так, інколи","Орієнтуюсь, приймав/ла участь в одній організації"
2,16-23 років,Жіноча,Студент,"Так, інколи","Не сильно, щось чув/ла, але не цікавився/лася."
3,16-23 років,Жіноча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась."


In [10]:
str(survey_data)

'data.frame':	40 obs. of  14 variables:
 $ X1 : chr  "16-23 років" "16-23 років" "16-23 років" "16-23 років" ...
 $ X2 : chr  "Жіноча" "Жіноча" "Жіноча" "Жіноча" ...
 $ X3 : chr  "Працюючий" "Студент" "Студент" "Студент" ...
 $ X4 : chr  "Так, інколи" "Так, інколи" "Так, інколи" "Так, часто" ...
 $ X5 : chr  "Орієнтуюсь, приймав/ла участь в одній організації" "Не сильно, щось чув/ла, але не цікавився/лася." "В середньому, ніколи не приймав/ла участь, але цікавився/лась." "В середньому, ніколи не приймав/ла участь, але цікавився/лась." ...
 $ X6 : chr  "Так" "Ні" "Так" "Ні" ...
 $ X7 : chr  "Молодіжна організація" "" "Молодіжна організація" "" ...
 $ X8 : chr  "" "" "" "" ...
 $ X9 : chr  "Ні" "Ні" "Ні" "Ні" ...
 $ X10: chr  "Так" "Ні" "Так" "Ні" ...
 $ X11: chr  "Так, декілька" "Ні, жодного" "Ні, жодного" "Так, декілька" ...
 $ X12: chr  "Творчі об'єднання" "Молодіжна організація" "Молодіжна організація" "Молодіжна організація" ...
 $ X13: chr  "Так, оскільки за допомогою них можна реа

Перетворимо відповіді з тексту до факторів та заміними пропуски на `"-"`.

In [13]:
survey_data <- survey_data |>
  mutate_all(list(~na_if(.,""))) |>
  mutate_if(is.character, ~replace_na(.,"-")) |>
  mutate_if(is.character, as.factor)
    
survey_data[5:10, 1:10]

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
5,16-23 років,Жіноча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Ні,-,Не мав/ла часу,Ні,Ні
6,16-23 років,Жіноча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Ні,-,Не мав/ла змоги,Ні,Так
7,16-23 років,Жіноча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Ні,-,Не мав/ла часу,Ні,Ні
8,16-23 років,Чоловіча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Ні,Молодіжна організація,Не мав/ла змоги,Ні,Так
9,16-23 років,Жіноча,Студент,"Так, інколи","В середньому, ніколи не приймав/ла участь, але цікавився/лась.",Так,Молодіжна організація,-,Ні,Ні
10,16-23 років,Жіноча,Студент,"Ні, ніколи","Не сильно, щось чув/ла, але не цікавився/лася.",Ні,-,Не цікаво,Ні,Так


Збережемо дані:

In [17]:
saveRDS(survey_data, "survey_data_init.rds")
saveRDS(survey_headers, "survey_headers.rds")